In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glob import glob
import numpy as np
import seaborn as sns
from pathlib import Path

In [3]:
process = 'PlotWholeBrain'
top_path = '/Users/emcmaho7/Dropbox/projects/SI_EEG/SIEEG_analysis'
input_path = f'{top_path}/data/interim'
raw_path = f'{top_path}/data/raw'
out_path = f'{top_path}/data/interim/{process}'
figure_path = f'{top_path}/reports/figures/{process}'
Path(f'{out_path}/combined_images').mkdir(parents=True, exist_ok=True)
Path(figure_path).mkdir(parents=True, exist_ok=True)

plotting_subj = 2
start, end = 0, 200
times = np.arange(-200, 1000, 2.5)

## fMRI Whole Brain

In [ ]:
import numpy as np
from nilearn import plotting, surface
import nibabel as nib
from src.tools import camera_switcher
import os
import warnings


os.environ["SUBJECTS_DIR"] = "/Users/emcmaho7/Dropbox/projects/SI_EEG/SIEEG_analysis/data/raw/freesurfer"
os.environ["FREESURFER_HOME"] = "/Applications/freesurfer"


def compute_surf_stats(prefix, sub, hemi):
    file = f'{prefix}_hemi-{hemi}.mgz'
    if not os.path.exists(file):
        cmd = '/Applications/freesurfer/bin/mri_vol2surf '
        cmd += f'--src {prefix}.nii.gz '
        cmd += f'--out {file} '
        cmd += f'--regheader sub-{sub} '
        cmd += f'--hemi {hemi} '
        cmd += '--projfrac 1 '
        cmd += '> /dev/null 2>&1'
        os.system(cmd)
    return surface.load_surf_data(file)


def load_surf_mesh(path, sub, hemi):
    return f'{path}/freesurfer/sub-{sub}/surf/{hemi}.inflated', \
            f'{path}/freesurfer/sub-{sub}/surf/{hemi}.sulc'


def plot_stats(surf_mesh, bg_map, surf_map, hemi_, figure_prefix,
               vmax=0.3, title=None, cmap_name='magma', 
               view='lateral', colorbar=False):
    cmap=sns.dark_palette(cmap_name, as_cmap=True)
    hemi_name = 'left' if hemi_ == 'lh' else 'right'

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="choosing both vmin and a threshold is not allowed; setting vmin to 0")
        fig = plotting.plot_surf_roi(surf_mesh=surf_mesh,
                                    roi_map=surf_map,
                                    bg_map=bg_map,
                                    vmax=vmax,
                                    vmin=0., 
                                    engine='plotly',
                                    colorbar=colorbar,
                                    view=view,
                                    cmap=cmap,
                                    title=title,
                                    title_font_size=50,
                                    hemi=hemi_name,
                                    kwargs={'symmetric_cmap': False})
        fig.figure.update_layout(coloraxis_colorbar=dict(
                                title='Explained variance ($r^2$)',
                                tickvals=[0, vmax],  # Positions of the ticks
                                ticktext=list(np.linspace(0, vmax, 4).round(2)),
                                thickness=25,
                                len=0.75,
                                x=1.02),
                                paper_bgcolor="rgba(0,0,0,0)",
                                plot_bgcolor="rgba(0,0,0,0)",
                                scene_camera=camera_switcher(hemi_, view))
        fig.figure.write_image(f'{figure_prefix}.png')

/Applications/anaconda3/envs/nibabel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
hemis = ['lh', 'rh']
views = ['ventral', 'medial', 'lateral']

subj_id = str(plotting_subj).zfill(2)
files = sorted(glob(f'{input_path}/fMRIWholeBrain/sub-{subj_id}_time*.nii.gz'))
for file, time in zip(files, times):
    if time >= start and time <= end: 
        stat_file = file.split('.')[0]
        plot_file_prefix = file.split('/')[-1].split('.')[0]
        plot_file_prefix += f'_timems-{time:.0f}'
        for hemi in hemis:
            surf = compute_surf_stats(stat_file, subj_id, hemi)
            surf[surf < 0] = 0.01
            inflated, sulcus = load_surf_mesh(raw_path, subj_id, hemi)
            for view in views:
                Path(f'{out_path}/{view}-{hemi}/').mkdir(exist_ok=True, parents=True)
                plot_file = f'{out_path}/{view}-{hemi}/{plot_file_prefix}'
                colorbar = True if hemi =='rh' and view == 'medial' else False
                plot_stats(inflated, sulcus, surf, hemi, plot_file,
                            cmap_name='#79C', colorbar=colorbar,
                            view=view, vmax=.35)

In [6]:
from PIL import Image
from PIL import Image, ImageDraw, ImageFont

def arrange_images_in_circle(image_paths, title, canvas_size=(1200, 650), resize_factor=0.6):
    # Create a new blank canvas
    canvas = Image.new('RGBA', canvas_size, (255, 255, 255, 0))

    # Calculate the angle between each image
    pos = {'lh': {'lateral': (int(canvas_size[0]*.15), int(canvas_size[1]*.01)),
                   'medial': (int(canvas_size[0]*.00015), int(canvas_size[1]*.39)),
                   'ventral': (int(canvas_size[0]*.15), int(canvas_size[1]*.39))}, 
           'rh': {'lateral': (int(canvas_size[0]*.5), int(canvas_size[1]*.01)),
                   'medial': (int(canvas_size[0]*.63025), int(canvas_size[1]*.39)),
                   'ventral': (int(canvas_size[0]*.5), int(canvas_size[1]*.39))}}

    # Load images and place them in a circle
    for hemi in ['rh', 'lh']:
        for view in ['lateral', 'medial', 'ventral']:
            for image_path in image_paths:
                if hemi in image_path and view in image_path: 
                    # Open the image
                    img = Image.open(image_path).convert('RGBA')

                    # Resize image if necessary
                    if view == 'ventral':
                        if hemi == 'rh': 
                            img = img.rotate(270, expand=True)
                        else: #hemi == 'lh':
                            img = img.rotate(90, expand=True)
                        ventral_factor = (resize_factor * 1.4)
                        new_size = (int(img.width * ventral_factor), int(img.height * ventral_factor))
                    else:
                        new_size = (int(img.width * resize_factor), int(img.height * resize_factor))
                    img = img.resize(new_size, Image.ANTIALIAS)

                    # Paste the image onto the canvas
                    canvas.paste(img, pos[hemi][view], img)

    # Convert to 'RGB' to remove alpha channel and replace transparency with white
    combined_image = Image.new('RGB', canvas_size, (255, 255, 255))
    combined_image.paste(canvas, (0, 0), canvas)

    # Add the time
    draw = ImageDraw.Draw(combined_image)
    font = ImageFont.truetype('~/Library/Fonts/Roboto/Roboto-Regular.ttf', 40)
    draw.text((int(canvas_size[0]*.45), int(canvas_size[1]*.01)),
              font=font, text=title, fill=(0, 0, 0))
    text_img = Image.new('RGBA', canvas_size, (255, 255, 255, 0))

    # Generate label for colorbar
    draw = ImageDraw.Draw(text_img)
    font = ImageFont.truetype('~/Library/Fonts/Roboto/Roboto-Regular.ttf', 16)
    draw.text((int(canvas_size[1]*.68), int(canvas_size[0]*.000001)),
              text='Prediction (r)', font=font, fill=(0, 0, 0))
    text_img = text_img.rotate(270, expand=False)

    # Combine into Image
    combined_image.paste(text_img, (270, 200), text_img)
    return combined_image

In [12]:
for itime, time in enumerate(np.arange(start, end+1, 2.5)):
    file_pattern = f'sub-{str(plotting_subj).zfill(2)}_*_timems-{time:.0f}.png'
    images = sorted(glob(os.path.join(f'{out_path}/*', file_pattern)))
    combined_image = arrange_images_in_circle(images, title=f'{time:.1f} ms')
    combined_image.save(f'{out_path}/combined_images/time-{str(itime).zfill(3)}.png')  # Save the final image

/var/folders/bp/2lmtx6qs4rzbksq18mtrbrjsfr_flb/T/ipykernel_27566/3855034908.py:34: DeprecationWarning:

ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.



In [13]:
import cv2
import re

def pngs_to_mp4(input_folder, output_file,
                file_pattern, fps=6, frame_size=None):
    """
    Converts a series of PNG images in a folder into an MP4 video file.

    Parameters:
    - input_folder: Path to the folder containing PNG images.
    - output_file: Path to save the MP4 video file.
    - fps: Frames per second in the output video.
    - frame_size: Tuple of (width, height) for the video frame size. If None, the size of the first image is used.
    """

    # Get all the PNG files in the folder
    images = sorted(glob(os.path.join(input_folder, file_pattern)))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
    if not frame_size:
        # If frame size is not specified, use the first image to determine the size
        first_image = cv2.imread(images[0])
        frame_size = (first_image.shape[1], first_image.shape[0])
    out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

    for image_path in images:
        img = cv2.imread(image_path)
        # Resize the image to match the frame size, if necessary
        if img.shape[1] != frame_size[0] or img.shape[0] != frame_size[1]:
            img = cv2.resize(img, frame_size)
        out.write(img)

    # Release everything when the job is finished
    out.release()
    cv2.destroyAllWindows()

In [15]:
pngs_to_mp4(input_folder=f'{out_path}/combined_images/',
            output_file=f'{out_path}/video.mp4',
            file_pattern='time-*.png')